In [ ]:
!git clone https://github.com/icedarold/through-pages.git
%cd through-pages


# SOTA Recommendation Pipeline: From Raw Data to Submission

This notebook runs the full pipeline:
1. **Preprocess State**: Clean interactions and items.
2. **Content Encoding (Phase 2)**: Multilingual embeddings.
3. **Multi-Interest (Phase 1)**: Train Transformer to extract 6 interest vectors.
4. **Feature Engineering (Phase 3)**: Generate rich features for 200 candidates.
5. **Reranking & Submission (Phase 4)**: Select Top 20.

In [ ]:
# 1. Install dependencies
!pip install -q sentence-transformers pyarrow fastparquet tqdm lightgbm

In [ ]:
# 2. Setup Directories (Paths updated for Kaggle Environment)
import os
OUT_DIR = '/kaggle/working/through-pages/experiments/data_v1'
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs('/kaggle/working/through-pages/experiments/models_v1', exist_ok=True)

print("Using Kaggle Input datasets: /kaggle/input/though-pages")

### Step 3: Global Preprocessing

In [ ]:
!python3 src/preprocess.py
!python3 src/data/items.py
!python3 src/data/sequences.py

### Step 4: Phase 2 - Item Content Embeddings
Generates 768-D vectors for all books.

In [ ]:
!python3 src/models/item_encoder.py --batch_size 128

### Step 5: Phase 1 - Multi-Interest Learning
Training the User Encoder to understand multi-faceted interests.

In [ ]:
!python3 src/data/enrich_items.py
!python3 src/train.py --epochs 30 --batch-size 256
!python3 src/inference_user.py

### Step 6: Phase 3 & 4 - SOTA Reranker
Preparing the Training Data and Gradient Boosting model (LightGBM).

In [ ]:
# 1. Create pairs for training reranker
!python3 src/data/make_reranker_train_data.py

# 2. Calculate features for Train pairs
!python3 src/data/feature_factory.py --mode train

# 3. Calculate features for Inference pairs (Candidates)
!python3 src/data/feature_factory.py --mode infer

# 4. Train LightGBM LambdaRank
!python3 src/models/train_reranker.py

### Step 7: Final Submission Generation

In [ ]:
!python3 src/submit.py --features $OUT_DIR/features_infer.parquet --output submission.csv

import pandas as pd
sub = pd.read_csv('submission.csv')
print("Submission generated successfully!")
display(sub.head(20))